In [1]:
# Importing library
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split
#Ref: https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/

In [82]:
data = pd.read_csv('https://raw.githubusercontent.com/ekamineni/DAEN690/main/Datarepo/Concat_dataset.csv')

In [84]:
data.shape

(1048575, 9)

In [85]:
data.head(10)

,obj1xPos,obj1yPos,obj1Dir,obj2xPos,obj2yPos,obj2Dir,objCxPos,objCyPos,count
0,1,2,0,7,1,1,1,0,20
1,9,8,7,4,2,1,7,6,20
2,1,1,1,1,0,7,3,1,20
3,7,6,7,6,8,2,5,4,20
4,1,1,2,7,9,0,7,1,10
5,0,8,4,3,7,1,1,8,20
6,7,8,5,0,5,0,5,8,20
7,1,7,2,7,3,0,7,7,10
8,6,0,0,8,2,7,6,2,20
9,9,8,3,5,4,1,7,8,20


In [86]:
#Spliting the data in train and test (70-30)

x_data = data.iloc[:,:6]
y_data = data.iloc[:,-3:]

X_train, X_test, Y_train, Y_test = train_test_split(x_data,y_data, test_size=0.3, random_state=12)

In [87]:
print(Y_test)

         objCxPos  objCyPos  count
236130          3         7     20
716473          3         1     17
203850          6         9      9
88225           2         1     20
1020415         3         2     20
...           ...       ...    ...
895720          5         2      0
958108          5         3      9
34398           4         1     20
587627          2         4     20
792538          1         1      6

[314573 rows x 3 columns]


In [116]:
# Checking how many are Colision points in testing dataset
Y_test[Y_test['count'] != 20]

,index,objCxPos,objCyPos,count
1,716473,3,1,17
2,203850,6,9,9
6,906283,3,7,7
7,724207,4,2,1
8,160220,1,1,8
...,...,...,...,...
314563,348188,3,6,8
314567,651788,7,3,9
314568,895720,5,2,0
314569,958108,5,3,9


In [88]:
# Importing Deep learning lib
from keras.layers import Dense
from keras.models import Sequential

In [89]:
#define the model

def get_model(input_number, output_number):
    model = Sequential()
    model.add(Dense(20, input_dim=input_number, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(20, input_dim=input_number, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(20, input_dim=input_number, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(20, input_dim=input_number, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(20, input_dim=input_number, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(output_number))
    model.compile(loss='mae', optimizer='adam')
    return model
    

In [90]:
# Fitting the model
result = list()
model = get_model(6,3)
model.fit(X_train,Y_train, verbose=0, epochs=100)


In [91]:
#Model MAE
mae= model.evaluate(X_test,Y_test, verbose=0)
mae

2.193267822265625

In [93]:
#Pred Value

Y_pred = model.predict(X_test)
Y_pred

array([[ 2.7963731 ,  6.3493805 , 19.830359  ],
       [ 0.45904785,  3.9439263 , 20.009174  ],
       [ 1.3773063 ,  2.1463242 , 19.941029  ],
       ...,
       [ 4.264889  ,  1.3887441 , 19.96138   ],
       [ 1.9603691 ,  3.895008  , 19.89603   ],
       [ 5.202989  ,  3.0509896 , 19.941532  ]], dtype=float32)

In [20]:
Y_test

,objCxPos,objCyPos,count
34622,3,8,20
1163,2,1,20
7637,3,7,20
7045,2,4,20
43847,6,0,20
...,...,...,...
6423,7,2,20
24349,0,0,20
2323,7,2,0
37804,2,4,20


In [99]:
# Converting into dataframe
actual_test_df = pd.DataFrame(Y_test)
actual_test_df

,objCxPos,objCyPos,count
236130,3,7,20
716473,3,1,17
203850,6,9,9
88225,2,1,20
1020415,3,2,20
...,...,...,...
895720,5,2,0
958108,5,3,9
34398,4,1,20
587627,2,4,20


In [100]:
import numpy as np
Y_pred = np.round(abs(Y_pred))
Y_pred

array([[ 3.,  6., 20.],
       [ 0.,  4., 20.],
       [ 1.,  2., 20.],
       ...,
       [ 4.,  1., 20.],
       [ 2.,  4., 20.],
       [ 5.,  3., 20.]], dtype=float32)

In [101]:
pred_test_df = pd.DataFrame(Y_pred, columns=['X_Collision','Y_Collision','Loop'])
pred_test_df

,X_Collision,Y_Collision,Loop
0,3.0,6.0,20.0
1,0.0,4.0,20.0
2,1.0,2.0,20.0
3,2.0,1.0,20.0
4,3.0,3.0,20.0
...,...,...,...
314568,5.0,2.0,0.0
314569,4.0,8.0,11.0
314570,4.0,1.0,20.0
314571,2.0,4.0,20.0


In [102]:
actual_test_df['count'].isin(pred_test_df['Loop']).value_counts()

True    314573
Name: count, dtype: int64

In [105]:
pred_test_df['Loop'].isin(actual_test_df['count']).value_counts()

True     313929
False       644
Name: Loop, dtype: int64

In [106]:

actual_test_df.reset_index(inplace=True)
actual_test_df.drop('index',axis=1,inplace=True)

In [108]:
actual_test_df

,objCxPos,objCyPos,count
0,3,7,20
1,3,1,17
2,6,9,9
3,2,1,20
4,3,2,20
...,...,...,...
314568,5,2,0
314569,5,3,9
314570,4,1,20
314571,2,4,20


In [109]:
pred_test_df.reset_index(inplace=True)
pred_test_df.drop('index',axis=1,inplace=True)

In [110]:
pred_test_df

,X_Collision,Y_Collision,Loop
0,3.0,6.0,20.0
1,0.0,4.0,20.0
2,1.0,2.0,20.0
3,2.0,1.0,20.0
4,3.0,3.0,20.0
...,...,...,...
314568,5.0,2.0,0.0
314569,4.0,8.0,11.0
314570,4.0,1.0,20.0
314571,2.0,4.0,20.0


In [117]:
# Logic for creating the confusion matrix

count_true_c = 0
count_true_nc = 0
count_false = 0
for i in range(actual_test_df.shape[0]):
    #print(i)
    
    if((actual_test_df.iloc[i][0] == pred_test_df.iloc[i][0]) & (actual_test_df.iloc[i][1] == pred_test_df.iloc[i][1]) & (actual_test_df.iloc[i][2] == pred_test_df.iloc[i][2]) & (actual_test_df.iloc[i][2] == 20)):
        count_true_nc += 1
    elif((actual_test_df.iloc[i][0] == pred_test_df.iloc[i][0]) & (actual_test_df.iloc[i][1] == pred_test_df.iloc[i][1]) & (actual_test_df.iloc[i][2] == pred_test_df.iloc[i][2]) & (actual_test_df.iloc[i][2] != 20)):
        count_true_c += 1
    else:
        count_false += 1
    

    
    

In [120]:
print(count_false)

229107


In [118]:
print(count_true_c)

13681


In [119]:
print(count_true_nc)

71785


In [123]:
# R square
from sklearn.metrics import r2_score
r2_score(actual_test_df,pred_test_df, multioutput='variance_weighted')

0.1669281455774178

In [60]:
from sklearn import metrics as m

In [62]:
from bokeh.plotting import figure, show, output_file


In [63]:
p = figure(title="Actual vs Predicted")
p.line(Y_test, legend_label='Actual Value')

TypeError: line() missing 1 required positional argument: 'y'

In [81]:
#m.multilabel_confusion_matrix(Y_test,Y_pred, labels=['a','b','c'])

In [80]:
#import numpy as np

#y_test_arg=np.argmax(Y_test,axis=1)
#Y_pred = np.argmax(model.predict(X_test),axis=1)
#print('Confusion Matrix')
#print(confusion_matrix(y_test_arg, Y_pred))